<a href="https://colab.research.google.com/github/Group-ProjectE17/PCA-Analysics/blob/limouts/Flex_Robot_Modeling_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

##Initilizations

In [1]:
import numpy as np
from itertools import combinations
from datetime import datetime, timedelta
#import datefinder
import pandas as pd
import string
import os
import os.path
import uuid
import random
from pprint import pprint
import json
import csv
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import linalg as lin
from sklearn import datasets
import sklearn
from sklearn.mixture import GaussianMixture
import smtplib
from sklearn.model_selection import train_test_split
import sklearn
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Function definitions

###Data analysis by Dr Maithreepaa

In [2]:
class mugas_data_analysis:
    def principle_component_values(self, modelDF, predictDF, nnPCs):
        tempModelDataPDF=modelDF.copy();
        tempPredictDataPDF=predictDF.copy();
        pca=sklearn.decomposition.PCA(n_components=nnPCs);
        modelnumPCA=pca.fit_transform(tempModelDataPDF);
        PCweights=pca.singular_values_;
        PCAmodelScore=pca.score(tempModelDataPDF);
        PCAmeans=pca.mean_
        predictnumPCA=pca.transform(tempPredictDataPDF);
        PCApredictScore=pca.score(tempPredictDataPDF);
    

        traceNames=[];
        for nn in range(len(PCweights)):
            traceNames+=['P'+str(nn)];

        mindex=tempModelDataPDF.index.values;
        pindex=tempPredictDataPDF.index.values;
        #figFactor=self.hist_plot(PPFactor.T,histBinSize, binRange, traceNames,'Distributions of the PC Factors','PC Factors','Frequency')
        pcaOutput={'PCA_object':pca,'PCAmeans':PCAmeans,'traceNames':traceNames, 'fitScores': {'PCApredictScore':PCApredictScore, 'PCAmodelScore':PCAmodelScore}, 'PCweights':PCweights, 'modelnumPCA':pd.DataFrame(modelnumPCA,index=mindex),'predictnumPCA':pd.DataFrame(predictnumPCA,index=pindex)}
        return pcaOutput


    def feature_extraction(self, modelDF, predictDF, nnPCs):
        tempModelDataPDF=modelDF.copy();
        tempPredictDataPDF=predictDF.copy();
        #nnPCs=min(len(tempModelDataPDF.index.values),len(tempModelDataPDF.columns));
        #nnPCs='mle'
        pca=sklearn.decomposition.PCA(n_components=nnPCs);
        modelnumPCA=pca.fit_transform(tempModelDataPDF);
        PCweights=pca.singular_values_;
        PCAmodelScore=pca.score(tempModelDataPDF);
        PCAmeans=pca.mean_
        predictnumPCA=pca.transform(tempPredictDataPDF);
        PCApredictScore=pca.score(tempPredictDataPDF);
        #scaling=sklearn.preprocessing.StandardScaler().fit(pd.DataFrame(numPCA));
        #scaledPCA=scaling.transform(numPCA);
        transformer = sklearn.decomposition.FactorAnalysis(n_components=nnPCs); # n_components=nnPCs, random_state=0);
        modelsvdFactor = transformer.fit_transform(tempModelDataPDF);
        FactorSVDmeans=transformer.mean_
        svdFactormodelScore=transformer.score(tempModelDataPDF);
        predictsvdFactor = transformer.transform(tempPredictDataPDF);
        svdFactorpredictScore=transformer.score(tempPredictDataPDF);
    

        traceNames=[];
        for nn in range(len(PCweights)):
            traceNames+=['P'+str(nn)];

        mindex=tempModelDataPDF.index.values;
        pindex=tempPredictDataPDF.index.values;
        #figFactor=self.hist_plot(PPFactor.T,histBinSize, binRange, traceNames,'Distributions of the PC Factors','PC Factors','Frequency')
        pcaOutput={'PCAmeans':PCAmeans,'FactorSVDmeans':FactorSVDmeans,'traceNames':traceNames, 'fitScores': {'PCApredictScore':PCApredictScore, 'PCAmodelScore':PCAmodelScore, 'svdFactormodelScore':svdFactormodelScore, 'svdFactorpredictScore':svdFactorpredictScore}, 'PCweights':PCweights, 'modelnumPCA':pd.DataFrame(modelnumPCA,index=mindex),'predictnumPCA':pd.DataFrame(predictnumPCA,index=pindex), 'modelsvdFactor':pd.DataFrame(modelsvdFactor,index=mindex),  'predictsvdFactor':pd.DataFrame(predictsvdFactor,index=pindex)}
        return pcaOutput

    def gmm_data_model(self, mData, nClusters):
        #mData - numeric centered np array of shape (ndata,nfactors)

        gmm = sklearn.mixture.GaussianMixture(n_components = nClusters)
        gmm.fit(mData)
        labels = gmm.predict(mData)
        clusterIC=[gmm.aic(mData), gmm.bic(mData)]
        print(clusterIC)
        fig1=md.scatter_plot3D(mData[:,0],mData[:,1],mData[:,2],labels)

        return {'gmm':gmm, 'clusterIC':clusterIC}


    def gmm_data_classifier(self, model_DataPDF, predict_DataPDF,nClusters):
        modelDataPDF0=model_DataPDF.copy();
        predictDataPDF0=predict_DataPDF.copy();
        modelData=modelDataPDF0.to_numpy();
        predictData=predictDataPDF0.to_numpy();
        nPCs=modelData.shape[1]
        #print([modelPCA[n].shape,predictPCA[n].shape])
        fitGMM=self.gmm_data_model(modelData,nClusters)
        gmm=fitGMM['gmm'];

        #Plabels=[];
        #for nn in range(nPCs):
        #    Plabels+=['PF'+str(nn)]

        mLabels = gmm.predict(modelData)
        #modelFitScore=gmm.score(modelData);
        modelDist={'Model Mean':gmm.means_, 'Model Covaraince':gmm.covariances_};
        modelSampleFitScore=gmm.score_samples(modelData);
        modelDataPDF0['labels']=mLabels
        modelDataPDF0['Fit Score']=modelSampleFitScore;

        pLabels = gmm.predict(predictData)
        #predictFitScore=gmm.score(predictData);
        predictProbDist=gmm.predict_proba(predictData);
        predictSampleFitScore=gmm.score_samples(predictData);

        predictDataPDF0['labels']=pLabels
        predictDataPDF0['Fit Score']=predictSampleFitScore;
        xx1=[]; yy1=[]; traceNames1=[];
        xx2=[]; yy2=[]; traceNames2=[];

        for kk in set(modelDataPDF0['labels'].to_list()):
            xx1+=[['All Data']];
            yy1+=[[len(modelDataPDF0[modelDataPDF0['labels']==kk])]];
            yy2+=[[len(predictDataPDF0[predictDataPDF0['labels']==kk])]];
            traceNames1+=['GP'+str(kk)]
        fig21=self.bar_plot(xx1,yy1,traceNames1,'Model Clusters','','Numbers','stack')
        fig31=self.bar_plot(xx1,yy2,traceNames1,'Predicted Clusters','','Numbers','stack')
        #print({'modelFitScore':modelFitScore, 'predictFitScore':predictFitScore})
        return {'labeled model data':modelDataPDF0, 'labeled predict data':predictDataPDF0,'fitScore':{'modelFitScore':modelSampleFitScore, 'predictFitScore':predictSampleFitScore},'Probability Model':modelDist}

    def logistic_reg_data_classifier(self, model_XDataPDF, model_YDataPDF, predict_XDataPDF):
        MXDataPDF=model_XDataPDF.copy();
        MYDataPDF=model_YDataPDF.copy();
        PXDataPDF=predict_XDataPDF.copy();
        clf = sklearn.linear_model.LogisticRegression(random_state=0).fit(MXDataPDF.to_numpy(), MYDataPDF.to_numpy());
        mLabels = clf.predict(MXDataPDF.to_numpy())
        modelDist=clf.predict_proba(MXDataPDF.to_numpy());
        modelSampleFitScore=clf.score(MXDataPDF.to_numpy(), MYDataPDF.to_numpy());
        MXDataPDF['labels']=mLabels
        mLabelTypes=list(set(mLabels));
        probLabels=[];
        for mlt in mLabelTypes:
            probLabels+=['p('+str(mlt)+')']
    
        MXDataPDF[probLabels]=modelDist;

        pLabels = clf.predict(PXDataPDF.to_numpy())
        predictDist=clf.predict_proba(PXDataPDF.to_numpy());
        pLabelTypes=set(probLabels);
        PXDataPDF['labels']=pLabels;
        pLabelTypes=list(set(pLabels));
        probLabels=[];
        for mlt in pLabelTypes:
            probLabels+=['p('+str(mlt)+')']

        PXDataPDF[probLabels]=predictDist;
        
        xx1=[]; yy1=[]; traceNames1=[];
        xx2=[]; yy2=[]; traceNames2=[];

        for kk in set(MXDataPDF['labels'].to_list()):
            xx1+=[['All Data']];
            yy1+=[[len(MXDataPDF[MXDataPDF['labels']==kk])]];
            yy2+=[[len(PXDataPDF[PXDataPDF['labels']==kk])]];
            traceNames1+=['GP-'+str(kk)]
        fig21=self.bar_plot(xx1,yy1,traceNames1,'Model Clusters','','Numbers','stack')
        fig31=self.bar_plot(xx1,yy2,traceNames1,'Predicted Clusters','','Numbers','stack')
        
        
        #print({'modelFitScore':modelFitScore, 'predictFitScore':predictFitScore})
        return {'labeled model data':MXDataPDF, 'labeled predict data':PXDataPDF,'Probability Model':{'modelDist':modelDist,'predictDist':predictDist}}


    def hist_plot(self, xx, binSize, binRange, traceNames, titleText, xLabel,yLabel):
        fig = go.Figure()

        for i,x in enumerate(xx):
            fig.add_trace(go.Histogram(
                x=x,
                #histnorm='percent',
                #cumulative_enabled=cumulative,
                name=traceNames[i], # name used in legend and hover labels
                xbins=dict( # bins used for histogram
                    start=binRange[0],
                    end=binRange[1],
                    size=binSize
                ),
                #marker_color='#EB89B5',
                opacity=0.75
            ))

        fig.update_layout(
            title_text=titleText, # title of plot
            xaxis_title_text=xLabel, # xaxis label
            yaxis_title_text=yLabel, # yaxis label
            bargap=0.2, # gap between bars of adjacent location coordinates
            bargroupgap=0.1 # gap between bars of the same location coordinates
        )

        fig.show()
        #fig.write_html("/content/drive/My Drive/LMS Logs/Course_STATS_upto_May/"+titleText+".html")
        return fig

    def scatter_plot3D(self,xx,yy,zz,color):
        fig = go.Figure(data=[go.Scatter3d(x=xx, y=yy, z=zz, 
                                    mode='markers',marker=dict(
          #size=12,
          color=color,                # set color to an array/list of desired values
          #colorscale='Viridis',   # choose a colorscale
          opacity=0.5
        ))])
        fig.show()
        return fig

    def bar_plot(self, xx, yy, traceName, figTitle, xLabel, yLabel, barmode):
        fig = go.Figure()
        for i,x in enumerate(xx):
            fig.add_trace(go.Bar(y=yy[i], x=x, name=traceName[i]))
        
        
        fig.update_layout(
            title=figTitle, xaxis_title=xLabel, yaxis_title=yLabel, barmode=barmode
            )
        fig.show()
        return fig   

In [11]:
def linear_regression_modeling(md,model_inputs,model_outputs,test_inputs,test_outputs):
    output_PCA=md.principle_component_values(model_outputs, test_outputs, len([*model_outputs]))
    model_Output_PCA=output_PCA['modelnumPCA']
    test_Output_PCA=output_PCA['predictnumPCA']
    input_PCA=md.principle_component_values(model_inputs, test_inputs, len([*model_inputs]))
    model_input_PCA=input_PCA['modelnumPCA']
    test_input_PCA=input_PCA['predictnumPCA']
    linear_reg_model=linear_reg_train(model_input_PCA,model_Output_PCA)
    predict_model_output_PCA=liner_reg_predict(model_input_PCA,linear_reg_model)
    model_PCA_r2_score=r2_score(model_Output_PCA, predict_model_output_PCA)
    #print(model_PCA_r2_score)
    predict_model_output  = output_PCA['PCA_object'].inverse_transform(predict_model_output_PCA)
    model_r2_score=r2_score(model_outputs, predict_model_output)
    #print(model_r2_score)
    predict_test_output_PCA=liner_reg_predict(test_input_PCA,linear_reg_model)
    test_PCA_r2_score=r2_score(test_Output_PCA, predict_test_output_PCA)
    #print(test_PCA_r2_score)
    predict_test_output  = output_PCA['PCA_object'].inverse_transform(predict_test_output_PCA)
    test_r2_score=r2_score(test_outputs, predict_test_output)
    print('R2 value = {}'.format(test_r2_score))

    return {'fitParameters':{'model_PCA_r2_score':model_PCA_r2_score,'model_r2_score':model_r2_score,'test_PCA_r2_score':test_PCA_r2_score,'test_r2_score':test_r2_score},'predict_model_output':predict_model_output,'predict_test_output':predict_test_output}

In [4]:
md=mugas_data_analysis()

###Other functions

In [5]:


def PCA_Analisis(input,output):
    pcaOutputData=md.principle_component_values(output,output,len([*output]))
    pcaInputData=md.principle_component_values(input,input,len([*input]))
    return [ pcaInputData,pcaOutputData]
    
def PCA_invers(pca_val):
    input=pca_val.to_numpy()
    pca = PCA(n_components=4)
    x_reconstructed  = pca.inverse_transform(input )
    return x_reconstructed



def reg_fit2_outputs(Xmodel,ymodel):
    regr = linear_model.LinearRegression()
    yregModel=regr.fit(Xmodel, ymodel)
    yregModelpredict=yregModel.predict(Xmodel)
    print(yregModel.coef_, 'Intercept: '+str(yregModel.intercept_))
    return yregModelpredict

def linear_reg_train(Xmodel,ymodel):
    regr = linear_model.LinearRegression()
    yregModel=regr.fit(Xmodel, ymodel)
    return yregModel

def liner_reg_predict(Xmodel,yregModel):
    yregModelpredict=yregModel.predict(Xmodel)
    return yregModelpredict
     

#Import data And Preprossesing

##Import Data

In [ ]:
csvFilePath='/content/drive/My Drive/Project_3rd_year/Data/{}'
columns=['Input1','Input2','Input3','Input4','Output1','Output2','Output3','Output4','Output5']
axial_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Axial_Data_Complete.csv'),header=None)
axial_data_frame.columns=columns
bending_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Bending_Data_Complete.csv'),header=None)
bending_data_frame.columns=columns
twisting_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Twist_Data_Complete.csv'),header=None)
twisting_data_frame.columns=columns

In [7]:
csvFilePath='/content/drive/My Drive/ME Dept/E17_Group_Project/{}'
columns=['Input1','Input2','Input3','Input4','Output1','Output2','Output3','Output4','Output5']
axial_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Axial_Data_Complete.csv'),header=None)
axial_data_frame.columns=columns
bending_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Bending_Data_Complete.csv'),header=None)
bending_data_frame.columns=columns
twisting_data_frame=pd.read_csv(csvFilePath.format('UnitCell_Twist_Data_Complete.csv'),header=None)
twisting_data_frame.columns=columns

##Data Split

In [8]:
#For Axial data
axial_data_model,axial_data_test=train_test_split(axial_data_frame,test_size=0.2)
#For Bending data
bending_data_model,bending_data_test=train_test_split(bending_data_frame,test_size=0.2)
#For Twist data
twisting_data_model,twisting_data_test=train_test_split(twisting_data_frame,test_size=0.2)

#Axial Data

##PCA_transform

In [7]:
factorType='modelnumPCA' #'modelsvdFactor' #'modelsvdFactor' [axial_data_frame['Input1']==-0.2]
model_inputs=axial_data_model[['Input1','Input2','Input3','Input4']]
model_outputs=axial_data_model[['Output2','Output3','Output4']] #,'Output5']]
[pcaInputData,pcaOutputData]=PCA_Analisis(model_inputs,model_outputs)

###Plots

In [24]:
figOutputPCA=md.hist_plot(pcaOutputData[factorType].to_numpy().T,0.5,[-100,100],pcaOutputData['traceNames'],'Distributions of the Output PCA','PCAs','Frequency')
figInputPCA=md.hist_plot(pcaInputData[factorType].to_numpy().T,0.0005,[-100,100],pcaInputData['traceNames'],'Distributions of the Input PCA','PCAs','Frequency')

In [ ]:
ydata=pcaOutputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=ydata[0].to_list(),y=ydata[2].to_list(),z=ydata[3].to_list(),mode='markers',marker={'size':2}))

In [ ]:
xdata=pcaInputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=xdata[3].to_list(),y=xdata[1].to_list(),z=xdata[2].to_list(),mode='markers',marker={'size':2}))

In [ ]:
ydata=pcaOutputData[factorType]
xdata=pcaInputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=xdata[0].to_list(),y=xdata[1].to_list(),z=ydata[0].to_list(),mode='markers',marker={'size':2}))

##Modeing

In [9]:
model_inputs=axial_data_model[['Input1','Input2','Input3','Input4']]
model_outputs=axial_data_model[['Output2','Output3','Output4']] #,'Output5']]
test_inputs=axial_data_test[['Input1','Input2','Input3','Input4']]
test_outputs=axial_data_test[['Output2','Output3','Output4']] #,'Output5']]

In [12]:
lin_reg_model=linear_regression_modeling(md,model_inputs,model_outputs,test_inputs,test_outputs)

R2 value = 0.9986449467964693


In [13]:
Xreal=test_outputs
Xpredict=lin_reg_model['predict_test_output']
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=Xreal['Output2'],y=Xreal['Output3'].to_list(),z=Xreal['Output4'].to_list(),mode='markers',marker={'size':2},name='Actual'))
fig.add_trace(go.Scatter3d(x=Xpredict[:,0],y=Xpredict[:,1],z=Xpredict[:,2],mode='markers',marker={'size':2},name='Predicted'))
fig.show()

#Bending data

In [ ]:
factorType='modelnumPCA' #'modelsvdFactor' #'modelsvdFactor' [axial_data_frame['Input1']==-0.2]

inputs=twisting_data_model[['Input1','Input2','Input3','Input4']]
outputs=twisting_data_model[['Output2','Output3','Output4','Output5']]

[pcaInputData,pcaOutputData]=PCA_Analisis(inputs,outputs)
figOutputPCA=md.hist_plot(pcaOutputData[factorType].to_numpy().T,0.5,[-100,100],pcaOutputData['traceNames'],'Distributions of the Output PCA','PCAs','Frequency')
figInputPCA=md.hist_plot(pcaInputData[factorType].to_numpy().T,0.0005,[-100,100],pcaInputData['traceNames'],'Distributions of the Input PCA','PCAs','Frequency')

In [ ]:
ydata=pcaOutputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=ydata[0].to_list(),y=ydata[1].to_list(),z=ydata[2].to_list(),mode='markers',marker={'size':2}))

In [ ]:
xdata=pcaInputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=xdata[3].to_list(),y=xdata[1].to_list(),z=xdata[2].to_list(),mode='markers',marker={'size':2}))

In [ ]:
ydata=pcaOutputData[factorType]
xdata=pcaInputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=xdata[0].to_list(),y=xdata[1].to_list(),z=ydata[0].to_list(),mode='markers',marker={'size':2}))

##Modeling

In [ ]:
Xmodel=pcaInputData[factorType][[0,1,2,3]]
for nn in [0,1,2,3]:
    y_n_model=pcaOutputData[factorType][nn]
    yregModelpredict=reg_fit2_outputs(Xmodel,y_n_model)
    fig=go.Figure()
    fig.add_trace(go.Scatter3d(x=Xmodel[2].to_list(),y=Xmodel[1].to_list(),z=y_n_model.to_list(),mode='markers',marker={'size':2}))
    fig.add_trace(go.Scatter3d(x=Xmodel[2].to_list(),y=Xmodel[1].to_list(),z=yregModelpredict,mode='markers',marker={'size':2}))
    fig.show()

#Twisting Data

In [ ]:
factorType='modelnumPCA' #'modelsvdFactor' #'modelsvdFactor' [axial_data_frame['Input1']==-0.2]

inputs=bending_data_model[['Input1','Input2','Input3','Input4']]
outputs=bending_data_model[['Output2','Output3','Output4','Output5']]
[ pcaInputData,pcaOutputData]=PCA_Analisis(inputs,outputs)
figOutputPCA=md.hist_plot(pcaOutputData[factorType].to_numpy().T,0.5,[-100,100],pcaOutputData['traceNames'],'Distributions of the Output PCA','PCAs','Frequency')
figInputPCA=md.hist_plot(pcaInputData[factorType].to_numpy().T,0.0005,[-100,100],pcaInputData['traceNames'],'Distributions of the Input PCA','PCAs','Frequency')

In [ ]:
ydata=pcaOutputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=ydata[0].to_list(),y=ydata[2].to_list(),z=ydata[3].to_list(),mode='markers',marker={'size':2}))

In [ ]:
xdata=pcaInputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=xdata[3].to_list(),y=xdata[1].to_list(),z=xdata[2].to_list(),mode='markers',marker={'size':2}))

In [ ]:
ydata=pcaOutputData[factorType]
xdata=pcaInputData[factorType]
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=xdata[0].to_list(),y=xdata[1].to_list(),z=ydata[0].to_list(),mode='markers',marker={'size':2}))

##Modeling

In [ ]:
Xmodel=pcaInputData[factorType][[0,1,2,3]]
for nn in [0,1,2,3]:
    y_n_model=pcaOutputData[factorType][nn]
    yregModelpredict=reg_fit2_outputs(Xmodel,y_n_model)
    fig=go.Figure()
    fig.add_trace(go.Scatter3d(x=Xmodel[2].to_list(),y=Xmodel[1].to_list(),z=y_n_model.to_list(),mode='markers',marker={'size':2}))
    fig.add_trace(go.Scatter3d(x=Xmodel[2].to_list(),y=Xmodel[1].to_list(),z=yregModelpredict,mode='markers',marker={'size':2}))
    fig.show()